## Capítulo 8 - Tratamento de dados: Junção, combinação e reformatação
    
Em muitas aplicações, os dados podem estar espalhados em vários arquivos ou banco de dados, ou podem estar organizados em um formato que não seja fácil de analisar. 

Inicialmente introduziremos o conceito de indexação hierárquica no pandas, a qual é intensamente utilizada em algumas dessas operações. Em seguida será explorada as manipulações de dados em particular.

### 8.1 Indexação hierárquica

A indexação hierárquica é um recurso importante do pandas; permite ter vários níveis de índices (dois ou mais) em um eixo. De forma, aé certo ponto, abstrata, oferece uma maneira de trabalhar com dados de dimensões mais altas em um formato de dimensões menores.

Vamos começar com um exemplo simples, criar uma Series com uma lista de listas (ou de arrays) como arrays:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9), 
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                      [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.674040
   2    2.637421
   3   -1.090238
b  1   -1.141084
   3   -0.659983
c  1   -0.138029
   2    0.231405
d  2    1.315703
   3    0.480536
dtype: float64

O que está vendo é uma visão elegante de uma Series com um MultiIndex como índice. As "lacunas" na exibição do índice significam "utilize o método imediatamente anterior":

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

Com um objeto hierarquicamente indexado, a chamada indexação parcial é possível, permitindo selecionar subconjuntos dos dados de forma concisa:

In [4]:
data['b']

1   -1.141084
3   -0.659983
dtype: float64

In [5]:
data['b':'c']

b  1   -1.141084
   3   -0.659983
c  1   -0.138029
   2    0.231405
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -1.141084
   3   -0.659983
d  2    1.315703
   3    0.480536
dtype: float64

###### A seleção é até mesmo possível a partir de um nível "mais interno":

In [7]:
data.loc[:, 2]

a    2.637421
c    0.231405
d    1.315703
dtype: float64

A indexação hierárquica desempenha um papel importante na reformatação dos dados e nas operações baseadas em grupos, como compor uma tabela pivô. 

Por exemplo, poderíamos reorganizar os dados em um DataFrame usando o seu método unstack:

In [8]:
data.unstack()

,1,2,3
a,0.674040,2.637421,-1.090238
b,-1.141084,NaN,-0.659983
c,-0.138029,0.231405,NaN
d,NaN,1.315703,0.480536


###### a operação inversa de unstack é stack:

In [9]:
data.unstack().stack()

a  1    0.674040
   2    2.637421
   3   -1.090238
b  1   -1.141084
   3   -0.659983
c  1   -0.138029
   2    0.231405
d  2    1.315703
   3    0.480536
dtype: float64

###### Em um DataFrame, qualquer eixo pode ter um índice hierárquico:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Os níveis hierárquicos podem ter nomes (como strings ou quaisquer objetos Python). Nesse caso, eles serão exibidos na saída do console:

In [11]:
frame.index.names = ['key1', 'key2']

In [12]:
frame.columns.names=['state', 'color']

In [13]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

###### Com uma indexação parcial de colunas, poderá, de modo semelhante, selecionar grupos de colunas:

In [14]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

Um MultIndex pode ser criado de modo independente e então ser reutilizado; as colunas no DataFrame anterior com nomes para os níveis poderiam ter sido criadas assim:

In [15]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], 
                     names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

#### Reorganizando e ordenando níveis 

Ocasionalmente precisaremos reorganizar a ordem dos níveis em um eixo ou ordenar os dados de acordo com os valores em um nível específico. 'swaplevel' aceita dois números ou nomes de níveis e devolve um novo objeto com os níveis trocados (os dados, porém, permanecem inalterados):

In [16]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

'sort-index', por outro lado, ordena os dados usando os valores de um só nível. Ao trocar níveis, não é incomum usar 'sort_index' também para que o resultado seja ordenado em ordem lexicográfica de acordo com o nível indicado:

In [17]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

#### Estatísticas de resumo por nível

Muitas estatísticas descritivas ou de resumo em DataFrame e em Series têm uma opção level; com ela, podemos especificar o nível de acordo com o qual queremos fazer uma agregação, em um eixo em particular. Considere o DataFrame anterior; podemos fazer uma agregação por nível, seja nas linhas ou nas colunas, da seguinte maneira:

In [19]:
frame.sum(level='key2')

/tmp/ipykernel_5344/2004046222.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='key2')


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.groupby(level='key2').sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [21]:
frame.sum(level='color', axis=1)

/tmp/ipykernel_5344/4133796543.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level='color', axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [22]:
frame.groupby(level='color', axis=1).sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#### Indexando com as colunas de um DataFrame

Não é incomum querer usar uma ou mais colunas de um DataFrame como índice de linha; de modo alternativo, talvez queira mover o índice das linhas para as colunas do DataFrame.

Eis um DataFrame de exemplo:

In [23]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


###### A função 'set_index' de DataFrame criará um novo DataFrame usando uma ou mais de suas colunas como índice:

In [24]:
frame2 = frame.set_index(['c', 'd'])

In [25]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

###### Por padrão, as colunas são removidas do DataFrame, embora possam ser mantidas:

In [26]:
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

###### 'reset_index', por outro lado, faz o inverso de 'set_index'; os níveis dos índices hierárquicos são passados para as colunas:

In [27]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 8.2 Combinando e mesclando conjunto de dados

Os dados contidos em objetos do pandas podem ser combinados de várias maneiras:
    
    -> pandas.merge => Conecta linhas em DataFrames com base em uma ou mais chaves. Essa operação será conhecida dos usuários de SQL ou de outros bancos de dados relacionais, pois implementa as operações de junção (join) dos bancos de dados.
    
    -> pandas.concat => Concatena ou 'empilha' objetos ao longo de um eixo.
    
    -> O método de instância 'combine_first' permite combinar dados que se sobrepõem a fim de preencher valores ausentes em um objeto com valores de outro objeto.

#### Junções no DataFrame no estilo de banco de dados

Operações de merge(mescla) ou de junção(join) combinam conjuntos de dados associando linhas por meio de uma ou mais chaves. Essas operações essenciais em banco de dados relacionais (por exemplo, naqueles baseados em SQL). A função merge do pandas é o ponto de entrada principal para usar esses algoritmos em seus dados.

###### Vamos começar com um exemplo simples:

In [28]:
df1 = pd.DataFrame({'key':['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [29]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


O exemplo a seguir é de uma junção de 'muitos-para-um' (many-to-one); os dados em df1 têm várias linhas de rótulos 'a' e 'b', enquanto df2 tem apenas uma linha para cada valor na coluna key. Se 'merge' for chamado nesses objetos, teremos o seguinte:

In [30]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Observe que não foi especificado a coluna para fazer a junção. Se essa informação não for especificada, 'merge' utilizará como chaves os nomes das colunas que se sobrepõem. No entanto, especificá-la explicitamente é uma boa prática:

In [31]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


###### Se os nomes das colunas forem diferentes em cada objeto, poderá especificá-las separadamente:

In [32]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'data1': range(7)})
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [33]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 
                    'data2': range(3)})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [34]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Talvez tenha percebido que os valores 'c' e 'd' e os dados associados estão ausentes no resultado. Por padrão o 'merge' executa uma junção do tipo 'inner'(interna); as chaves no resultado são a intersecção, ou o conjunto comum que se encontra nas duas tabelas. Outras opções possíveis são: 'left', 'right' e 'outer'. A junção externa (outer join) efetua a união das chaves, combinando o efeito da aplicação das junções tanto à esquerda quanto à direita:

In [35]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


###### Tabela 8.1 - Diferentes tipos de junção com o argumento how

opção 

'inner' => Utiliza somente as combinações de chaves observadas nas duas tabelas;

'left' => Utiliza todas as combinações de chaves encontradas na tabela à esquerda;

'right' => Utiliza todas as combinações de chaves encontradas da tabela à direita;

'outer' => Utiliza todas as combinações de chaves observadas nas duas tabelas em conjunto.

Merges de 'muitos-para-muitos' (many-to-many) têm um comportamento bem definido, embora não seja necessariamente intuitivo.

Eis um exemplo:

In [36]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [37]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'a'], 
                    'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,a,4


In [38]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,a,2,4.0
7,c,3,NaN
8,a,4,0.0
9,a,4,2.0


Junções de 'muitos para muitos' formam o produto cartesiano das linhas. Como havia três linhas 'b' no DataFrame da esquerda e duas no da direita, há seis linhas 'b' no resultado. O método de junção afeta somente os valores de chaves distintos que aparecem no resultado:

In [39]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,2,4
9,a,4,0


###### Para um merge com várias chaves, passe uma lista de nomes de coluna:

In [40]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'], 
                     'key2': ['one', 'two', 'one'], 
                     'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [41]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                      'key2': ['one', 'one', 'one', 'two'], 
                      'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [42]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


Para determinar quais combinações de chaves aparecerão no resultado conforme a escolha do método de merge, pense nas várias chaves como compondo um array de tuplas a ser usado como uma única chave de junção (apesar de isso não ser implementado dessa forma).

Ao fazer a junção de 'colunas-sobre-colunas', os índices dos objetos DataFrame especificados serão descartados.

Última questão a ser considerada em operações de merge é o tratamento dos nomes de coluna que se sobrepõem. Embora seja possível tratar a sobreposição manualmente, merge tem uma opção 'suffixes' para especificar strings a serem concatenadas nos nomes que se sobrepõem, nos objetos DataFrame à esquerda e à direita:

In [43]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [44]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


###### Tabela 8.2 - Argumentos da função merge

Argumento 

-> left => DataFrame à esquerda para o merge.

-> right => DataFrame à direita para o merge.

-> how => Uma opção entre 'inner', 'outer', 'left' ou 'right'; o default é 'inner'.

-> on => Nomes de coluna para a junção. Deve ser encontrado nos dois objetos DataFrame. Se não for especificado e nenhuma outra chave de junção for definida, usará a intersecção entre os nomes de colunas em left e em right como as chaves da junção.

-> left_on => Colunas no DataFrame 'left' a serem usadas como chaves de junção.

-> right_on => Análogo ao 'left_on' para o DataFrame right.

-> left_index => Utiliza o índice de linha em left como sua chave de junção (ou chaves, se for um MultIndex).

-> right_index => Análogo a 'left_index'.

-> sort => Ordena os dados do merge em ordem lexicográfica, de acordo com as chaves de junção; o default é True (desative-o para ter um desempenho melhor em alguns casos, quando os conjuntos de dados forem grandes). 

-> suffixes => Tupla de valores de strings a serem concatenados aos nomes de colunas em caso de sobreposição; o default('_x', '_y')(por exemplo, se houver 'data' nos dois objetos DataFrame, esses aparecerão como 'data_x' e 'data_y' no resultado).

-> copy => Se for False, evita copiar dados para a estrutura de dados resultante em alguns casos excepcionais; por padrão, sempre copia.

-> indicator => Adiciona uma coluna especial '_merge' que informa a origem de cada linha; os valores serão 'left_only', 'right_only' ou 'both' conforme a origem dos dados da junção em cada linha.

#### Fazendo merge com base no índice

Em alguns casos, a(s) chave(s) do merge em um DataFrame serão encontradas em seu índice. Nessa situação, poderá passar 'left_index=True' ou 'right_index=True' (ou ambos) para informar que o índice deverá ser usado como a chave do merge:

In [45]:
left1 = pd.DataFrame({'key1': ['a', 'b', 'a', 'a', 'b', 'c'], 
                      'value': range(6)})
left1

,key1,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [46]:
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [47]:
pd.merge(left1, right1, left_on='key1', right_index=True)

,key1,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Como o método default de merge consiste em fazer uma intersecção das chaves de junção, poderá, de modo alternativo, compor a união dels fazendo uma junção externa:

In [48]:
pd.merge(left1, right1, left_on='key1', right_index=True, how='outer')

,key1,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Com dados hierarquicamente indexados, a situação é mais complicada, pois fazer uma junção com base no índice é implicitamente um merge de várias chaves:

In [49]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
                      'key2': [2000, 2001, 2002, 2001, 2002], 
                      'data': np.arange(5.)})
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [50]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)), 
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'], 
                             [2001, 2000, 2000, 2000, 2001, 2002]], 
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

Nesse caso, deverá especificar várias colunas com base nas quais o merge será feito, na forma de uma lista (observe o tratamento de valores de índice duplicados com how='outer'):

In [51]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [52]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


###### Usar os índices dos dois lados do merge também é possível:

In [53]:
left2 = pd.DataFrame([[1.,2.], [3., 4.], [5., 6.]], 
                     index=['a', 'c', 'e'], 
                     columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [54]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                     index=['b', 'c', 'd', 'e'], 
                     columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [55]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


O DataFrame tem uma instância conveniente de 'join' para fazer o merge pelo índice. Ela também pode ser usada para combinar vários objetos DataFrame com índices iguais ou semelhantes, porém com colunas que não se sobrepõem. 

###### No exemplo anterior poderíamos ter escrito:

In [56]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Em parte por questões de legado (isto é, versões bem antigas do pandas), o método 'join' do DataFrame realiza uma junção à esquerda nas chaves de junção, preservando exatamente  o índice de linhas do frame à esquerda. Ele também aceita a junção do índice do DataFrame recebido em uma das colunas do DataFrame que faz a chamada:

In [57]:
left1.join(right1, on='key1')

,key1,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Por fim, para merges simples de índice-sobre-índice, podemos passar uma lista de DataFrames para 'join' como uma alternativa ao uso da função 'concat', mais genérica.

In [58]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], 
                       index=['a', 'c', 'e', 'f'], 
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [59]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [60]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


#### Concatenando ao longo de um eixo

Outro tipo de operação de combinação de dados é chamado, de forma indistinta, de concatenação, vinculação (binding) ou empilhamento (stacking). A função 'concatenate' do NumPy é capaz de fazer isso com arrays NumPy:

In [61]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [62]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

No contexto de objetos do pandas, como Series e DataFrame, ter eixos com rótulos permite generalizar melhor a concatenação de arrays. Em particular, há um número adicional de aspectos nos quais devemos pensar:
    
    -> Se os objetos estiverem indexados de modo diferente nos outros eixos, devemos combinar os elementos distintos nesses eixos ou usar somente os valores compartilhados (a intersecção)?
    
    -> As porções de dados concatenados devem ser identificáveis no objeto resultante?
    
    -> O "eixo de concatenação" contém dados que devam ser preservados? Em muitos casos, será melhor que os rótulos inteiros default em um DataFrame sejam descartados durante a concatenação.

A função 'concat' do pandas oferece uma forma consistente de tratar cada um desses aspectos. Será apresentada uma série de exemplos para mostrar o seu funcionamento.

Suponha que tenhamos três Series sem sobreposição de índices:

In [63]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [64]:
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [65]:
s3 = pd.Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

In [66]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

###### Por padrão, 'concat' atua em axis=0, gerando outra Series. Se passar axis=1, o resultado será um DataFrame (axis=1 são as colunas)

In [67]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Nesse caso, não há sobreposição no outro eixoque, como podemos ver, é a união ordenada (a junção 'outer') do índices. Por outro lado, podemos fazer a sua intersecção passando join='inner':

In [68]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [69]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [70]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


Um possível problema é o fato de as partes concatenadas não serem identificadas não serem identificáveis no resultado. Suponha que quiséssemos criar um índice hierárquico no eixo da concatenação. Para isso, utilize o argumento 'keys':

In [71]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [72]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


###### No caso da combinação de Series ao longo de axis=1, as keys passam a ser os cabeçalhos das colunas do DataFrame:

In [73]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


###### a mesma lógica se estende aos objetos DataFrame:

In [74]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], 
                   columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [75]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], 
                   columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [76]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Há argumentos adicionais que determinam o modo como o índice hierárquico é criado (veja a Tabela 8.3).Por exemplo, podemos nomear os níveis criados no eixo com o argumento 'names':

In [77]:
pd.concat([df1, df2], axis=1, keys=['keys1', 'keys2'], 
          names=['upper', 'lower'])

upper keys1     keys2     
lower   one two three four
a         0   1   5.0  6.0
b         2   3   NaN  NaN
c         4   5   7.0  8.0

###### Uma última consideração diz respeito aos DataFrames em que o índice das linhas não contém nenhum dado relevante:

In [78]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,-1.071960,-0.513574,-0.831749,0.016659
1,0.366006,-0.403743,0.807793,-0.296708
2,-0.874547,0.748416,0.134410,-1.398528


In [79]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,1.670479,-0.705171,-0.854238
1,-0.012569,-0.205501,-0.741865


###### Nesse caso, podemos passar 'ignore_index=True':

In [80]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-1.071960,-0.513574,-0.831749,0.016659
1,0.366006,-0.403743,0.807793,-0.296708
2,-0.874547,0.748416,0.134410,-1.398528
3,-0.854238,1.670479,NaN,-0.705171
4,-0.741865,-0.012569,NaN,-0.205501


###### Tabela 8.3 - Argumentos da função concat

Argumento

-> objs => Lista ou dicionário de objetos do pandas a serem concatenados; é o único argumento obrigatório.

-> axis => Eixo no qual será feita a concatenação; o default é 0 (nas linhas).

-> join => Pode ser 'inner' ou 'outer' (o default é 'outer'); indica se será feita a intersecção (inner) ou a união (outer) do índices ao longo dos  outros eixos.

-> keys => Valores a serem associados aos objetos sendo concatenados, compondo um índice hierárquico ao longo do eixo de concatenação; pode ser uma lista ou um array de valores arbitrários, um array de tuplas ou uma lista de arrays (se arrays de vários níveis forem especificados em 'levels').

-> levels => Índices específicos a serem usados como nível ou níveis de índices hierárquicos se chaves forem especificadas.

-> names => Nomes para os níveis hierárquicos criados se 'keys' e/ou 'levels' forem especificados.

-> verifiy_integrity => Verifica o novo eixo no objeto concatenado em busca de duplicatas e lança uma exceção em caso afirmativo; por padrão(False), permite duplicatas.

-> ignore_index => Não preserva os índices ao longo do 'axis' de concatenação, gerando um novo índice range(total_length) em seu lugar.

#### Combinando dados com subreposição

Há outra situação de combinação de dados que não pode ser expressa nem com uma operação de merge nem como de concatenação. Podemos ter dois conjuntos de dados, cujos índices se sobreponham de forma total ou parcial. Como um exemplo motivador, considere a função 'where' do NumPy, que executa o equivalente a uma expressão if-else, porém orientado a arrays:

In [81]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [82]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=['a', 'b', 'c', 'd', 'e', 'f'])
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [83]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

###### Uma Series tem um método 'combine_first', que executa o equivalente a essa operação, junto com a lógica usual de alinhamento de dados do pandas:

In [84]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

Com DataFrames, 'combine_first' faz o mesmo coluna a coluna, portanto podemos pensar nele como se estivesse fazendo um 'patching' dos dados ausentes no objeto que faz a chamada, com os dados do objeto que lhe passar:

In [85]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan], 
                    'b': [np.nan, 2., np.nan, 6.], 
                    'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [86]:
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 
                    'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [87]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 8.3 Reformatação e pivoteamento

Há uma série de operações básicas para reorganização de dados tabulares. De modo alternativo, são chamadas de operações de 'reformatação'(reshaping) ou 'pivoteamento'(pivoting).

#### Reformatação com indexação hierárquica

A indexação hierárquica oferece uma forma consistente de reorganizar dados em um DataFrame.

###### Há duas ações principais:

-> stack => Faz a 'rotação' ou o pivoteamento das colunas dos dados para as linhas.

-> unstack => Faz o pivoteamento das linhas para as colunas.

Será demonstratado essas operações com uma série de exemplos. Considere um pequeno DataFrame com arrays de strings como índices das linhas e das colunas:

In [88]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='state'), 
                    columns=pd.Index(['one', 'two', 'three'], 
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


###### Utilizar o método 'stack' nesses dados faz o pivoteamento das colunas para as linhas gerando uma Series:

In [89]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

###### A partir de uma Series indexada hierarquicamente, podemos reorganizar os dados de volta em um DataFrame usando 'unstack':

In [90]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


###### Por padrão, o nível mais interno será desempilhado (o mesmo vale para stack).Podemos desempilhar um nível diferente passando um número ou o nome de um nível:

In [91]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [92]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


###### Desempilhar pode introduzir dados ausentes se nem todos os valores do nível forem encontrados em cada um dos subgrupos:

In [93]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s1

a    0
b    1
c    2
d    3
dtype: int64

In [94]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s2

c    4
d    5
e    6
dtype: int64

In [95]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [96]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


###### O empilhamento filtra dados ausentes por padrão, de modo que a operação seja mais facilmente revertida:

In [97]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [98]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [99]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

###### Quando desempilhamos dados em um DataFrame, o nível desempilhado passa a ser o nível mais baixo no resultado:

In [100]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [101]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

###### Quando chamamos 'stack', podemos informar o nome do eixo a ser empilhado:

In [102]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

#### Fazendo o pivoteamento de um formato "longo" para um formato "largo"

Uma maneira comum de armazenar várias séries temporais em bancos de dados e em CSV é usar o chamado formato longo (long) ou empilhado (stacked). Vamos carregar alguns dados de exemplo e fazer uma pequena manipulação de séries temporais e outras operações de limpeza de dados:

In [103]:
data = pd.read_csv('examples/macrodata.csv')

In [104]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [105]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [106]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [107]:
data.index = periods.to_timestamp('D', 'end')

In [108]:
ldata = data.stack().reset_index().rename(columns={0: 'value', 'level_1':'item'})
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,year,1959.000
1,1959-03-31 23:59:59.999999999,quarter,1.000
2,1959-03-31 23:59:59.999999999,realgdp,2710.349
3,1959-03-31 23:59:59.999999999,realcons,1707.400
4,1959-03-31 23:59:59.999999999,realinv,286.898
...,...,...,...
2837,2009-09-30 23:59:59.999999999,tbilrate,0.120
2838,2009-09-30 23:59:59.999999999,unemp,9.600
2839,2009-09-30 23:59:59.999999999,pop,308.013
2840,2009-09-30 23:59:59.999999999,infl,3.560


###### 'PeriodIndex' = Combina as colunas 'year' e 'quarter' de modo a criar um tipo de intervalo de tempo:

In [109]:
# Eis a aparência do ldata:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,year,1959.000
1,1959-03-31 23:59:59.999999999,quarter,1.000
2,1959-03-31 23:59:59.999999999,realgdp,2710.349
3,1959-03-31 23:59:59.999999999,realcons,1707.400
4,1959-03-31 23:59:59.999999999,realinv,286.898
5,1959-03-31 23:59:59.999999999,realgovt,470.045
6,1959-03-31 23:59:59.999999999,realdpi,1886.900
7,1959-03-31 23:59:59.999999999,cpi,28.980
8,1959-03-31 23:59:59.999999999,m1,139.700
9,1959-03-31 23:59:59.999999999,tbilrate,2.820


Esse é o chamado formato longo para várias séries temporais, ou outros dados observados com duas ou mais chaves (nesse caso, nossas chaves são a data e o item). Cada linha da tabela representa uma única observação.

Os dados são frequentemente armazenados dessa maneira em banco de dados relacionais como o MySQL, pois um esquema fixo (nomes de colunas e tipos de dados) permite que o número de valores distintos na coluna 'item' mude á medida que os dados são adicionados à tabela. No exemplo anterior, 'date' e 'item' normalmente seriam as chaves primárias, oferecendo tanto uma integridade relacional quanto junções mais fáceis. Em alguns casos, talvez seja mais difícil trabalhar com os dados nesse formato; pode preferir ter um DataFrame contendo uma coluna por valor distinto de 'item' indexado por timestamps na coluna 'date'. O método 'pivot' de DataFrame executa exatamente essa transformação:

In [110]:
ldata
pivoted = ldata.pivot(index='date', columns='item', values='value')
pivoted

item,cpi,infl,m1,pop,quarter,realcons,realdpi,realgdp,realgovt,realint,realinv,tbilrate,unemp,year
date,,,,,,,,,,,,,,
1959-03-31 23:59:59.999999999,28.980,0.00,139.7,177.146,1.0,1707.4,1886.9,2710.349,470.045,0.00,286.898,2.82,5.8,1959.0
1959-06-30 23:59:59.999999999,29.150,2.34,141.7,177.830,2.0,1733.7,1919.7,2778.801,481.301,0.74,310.859,3.08,5.1,1959.0
1959-09-30 23:59:59.999999999,29.350,2.74,140.5,178.657,3.0,1751.8,1916.4,2775.488,491.260,1.09,289.226,3.82,5.3,1959.0
1959-12-31 23:59:59.999999999,29.370,0.27,140.0,179.386,4.0,1753.7,1931.3,2785.204,484.052,4.06,299.356,4.33,5.6,1959.0
1960-03-31 23:59:59.999999999,29.540,2.31,139.6,180.007,1.0,1770.5,1955.5,2847.699,462.199,1.19,331.722,3.50,5.2,1960.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-09-30 23:59:59.999999999,216.889,-3.16,1474.7,305.270,3.0,9267.7,9838.3,13324.600,991.551,4.33,1990.693,1.17,6.0,2008.0
2008-12-31 23:59:59.999999999,212.174,-8.79,1576.5,305.952,4.0,9195.3,9920.4,13141.920,1007.273,8.91,1857.661,0.12,6.9,2008.0
2009-03-31 23:59:59.999999999,212.671,0.94,1592.8,306.547,1.0,9209.2,9926.4,12925.410,996.287,-0.71,1558.494,0.22,8.1,2009.0


Os dois primeiros valores especificados são as colunas a serem usadas respectivamente como o índice das linhas e das colunas e, então, por fim, um valor de coluna opcional para preencher o DataFrame. Suponha que tivéssemos dois valores de coluna que quiséssemos reformatar simultaneamente:

In [111]:
ldata['value2'] = np.random.randn(len(ldata))

In [112]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,year,1959.000,0.907554
1,1959-03-31 23:59:59.999999999,quarter,1.000,-0.383996
2,1959-03-31 23:59:59.999999999,realgdp,2710.349,2.087139
3,1959-03-31 23:59:59.999999999,realcons,1707.400,0.510911
4,1959-03-31 23:59:59.999999999,realinv,286.898,-0.610097
5,1959-03-31 23:59:59.999999999,realgovt,470.045,0.328297
6,1959-03-31 23:59:59.999999999,realdpi,1886.900,-0.664446
7,1959-03-31 23:59:59.999999999,cpi,28.980,0.090678
8,1959-03-31 23:59:59.999999999,m1,139.700,0.005073
9,1959-03-31 23:59:59.999999999,tbilrate,2.820,0.566738


###### Ao omitir o último argumento, obteremos um DataFrame com colunas hierárquicas:

In [113]:
pivoted = ldata.pivot('date', 'item')

In [114]:
pivoted[:5]

value                                         \
item                             cpi  infl     m1      pop quarter realcons   
date                                                                          
1959-03-31 23:59:59.999999999  28.98  0.00  139.7  177.146     1.0   1707.4   
1959-06-30 23:59:59.999999999  29.15  2.34  141.7  177.830     2.0   1733.7   
1959-09-30 23:59:59.999999999  29.35  2.74  140.5  178.657     3.0   1751.8   
1959-12-31 23:59:59.999999999  29.37  0.27  140.0  179.386     4.0   1753.7   
1960-03-31 23:59:59.999999999  29.54  2.31  139.6  180.007     1.0   1770.5   

                                                                  ...  \
item                          realdpi   realgdp realgovt realint  ...   
date                                                              ...   
1959-03-31 23:59:59.999999999  1886.9  2710.349  470.045    0.00  ...   
1959-06-30 23:59:59.999999999  1919.7  2778.801  481.301    0.74  ...   
1959-09-30 23:59:59.999999999  1916.4  2775.488  491.260    1.09  ...   
1959-12-31 23:59:59.999999999  1931.3  2785.204  484.052    4.06  ...   
1960-03-31 23:59:59.999999999  1955.5  2847.699  462.199    1.19  ...   

                                 value2                                \
item                            quarter  realcons   realdpi   realgdp   
date                                                                    
1959-03-31 23:59:59.999999999 -0.383996  0.510911 -0.664446  2.087139   
1959-06-30 23:59:59.999999999  2.947663 -0.048073  0.036719  0.077935   
1959-09-30 23:59:59.999999999 -0.261806  1.382444 -1.346429  0.717486   
1959-12-31 23:59:59.999999999 -1.051768 -1.159161  0.296767 -1.494816   
1960-03-31 23:59:59.999999999  1.521940 -0.674328 -0.046744  1.017846   

                                                                       \
item                           realgovt   realint   realinv  tbilrate   
date                                                                    
1959-03-31 23:59:59.999999999  0.328297  1.716333 -0.610097  0.566738   
1959-06-30 23:59:59.999999999  1.018242  1.263644 -0.187125  0.875910   
1959-09-30 23:59:59.999999999  0.420715  1.292787  1.291063  0.986870   
1959-12-31 23:59:59.999999999  1.342230  1.377795 -0.414471 -1.745205   
1960-03-31 23:59:59.999999999  0.401212  1.016817  0.656887  0.412131   

                                                   
item                              unemp      year  
date                                               
1959-03-31 23:59:59.999999999 -1.683804  0.907554  
1959-06-30 23:59:59.999999999 -0.112548  2.033115  
1959-09-30 23:59:59.999999999 -0.767563 -0.901695  
1959-12-31 23:59:59.999999999  0.414969 -0.133802  
1960-03-31 23:59:59.999999999 -0.667184  0.172803  

[5 rows x 28 columns]

In [115]:
pivoted['value'][:5]

item,cpi,infl,m1,pop,quarter,realcons,realdpi,realgdp,realgovt,realint,realinv,tbilrate,unemp,year
date,,,,,,,,,,,,,,
1959-03-31 23:59:59.999999999,28.98,0.00,139.7,177.146,1.0,1707.4,1886.9,2710.349,470.045,0.00,286.898,2.82,5.8,1959.0
1959-06-30 23:59:59.999999999,29.15,2.34,141.7,177.830,2.0,1733.7,1919.7,2778.801,481.301,0.74,310.859,3.08,5.1,1959.0
1959-09-30 23:59:59.999999999,29.35,2.74,140.5,178.657,3.0,1751.8,1916.4,2775.488,491.260,1.09,289.226,3.82,5.3,1959.0
1959-12-31 23:59:59.999999999,29.37,0.27,140.0,179.386,4.0,1753.7,1931.3,2785.204,484.052,4.06,299.356,4.33,5.6,1959.0
1960-03-31 23:59:59.999999999,29.54,2.31,139.6,180.007,1.0,1770.5,1955.5,2847.699,462.199,1.19,331.722,3.50,5.2,1960.0


###### Observe que 'pivot' é equivalente a criar um índice hierárquico usando 'set_index' seguido de uma chamada a unstack:

In [116]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

In [117]:
unstacked[:7]

value                                         \
item                             cpi  infl     m1      pop quarter realcons   
date                                                                          
1959-03-31 23:59:59.999999999  28.98  0.00  139.7  177.146     1.0   1707.4   
1959-06-30 23:59:59.999999999  29.15  2.34  141.7  177.830     2.0   1733.7   
1959-09-30 23:59:59.999999999  29.35  2.74  140.5  178.657     3.0   1751.8   
1959-12-31 23:59:59.999999999  29.37  0.27  140.0  179.386     4.0   1753.7   
1960-03-31 23:59:59.999999999  29.54  2.31  139.6  180.007     1.0   1770.5   
1960-06-30 23:59:59.999999999  29.55  0.14  140.2  180.671     2.0   1792.9   
1960-09-30 23:59:59.999999999  29.75  2.70  140.9  181.528     3.0   1785.8   

                                                                  ...  \
item                          realdpi   realgdp realgovt realint  ...   
date                                                              ...   
1959-03-31 23:59:59.999999999  1886.9  2710.349  470.045    0.00  ...   
1959-06-30 23:59:59.999999999  1919.7  2778.801  481.301    0.74  ...   
1959-09-30 23:59:59.999999999  1916.4  2775.488  491.260    1.09  ...   
1959-12-31 23:59:59.999999999  1931.3  2785.204  484.052    4.06  ...   
1960-03-31 23:59:59.999999999  1955.5  2847.699  462.199    1.19  ...   
1960-06-30 23:59:59.999999999  1966.1  2834.390  460.400    2.55  ...   
1960-09-30 23:59:59.999999999  1967.8  2839.022  474.676   -0.34  ...   

                                 value2                                \
item                            quarter  realcons   realdpi   realgdp   
date                                                                    
1959-03-31 23:59:59.999999999 -0.383996  0.510911 -0.664446  2.087139   
1959-06-30 23:59:59.999999999  2.947663 -0.048073  0.036719  0.077935   
1959-09-30 23:59:59.999999999 -0.261806  1.382444 -1.346429  0.717486   
1959-12-31 23:59:59.999999999 -1.051768 -1.159161  0.296767 -1.494816   
1960-03-31 23:59:59.999999999  1.521940 -0.674328 -0.046744  1.017846   
1960-06-30 23:59:59.999999999 -0.066698 -0.904107 -0.760333  0.353409   
1960-09-30 23:59:59.999999999 -1.143599  1.108398  0.932145 -0.134519   

                                                                       \
item                           realgovt   realint   realinv  tbilrate   
date                                                                    
1959-03-31 23:59:59.999999999  0.328297  1.716333 -0.610097  0.566738   
1959-06-30 23:59:59.999999999  1.018242  1.263644 -0.187125  0.875910   
1959-09-30 23:59:59.999999999  0.420715  1.292787  1.291063  0.986870   
1959-12-31 23:59:59.999999999  1.342230  1.377795 -0.414471 -1.745205   
1960-03-31 23:59:59.999999999  0.401212  1.016817  0.656887  0.412131   
1960-06-30 23:59:59.999999999  0.456555 -0.996288 -0.403755  0.487678   
1960-09-30 23:59:59.999999999  2.365999  0.991591  0.362607 -0.785699   

                                                   
item                              unemp      year  
date                                               
1959-03-31 23:59:59.999999999 -1.683804  0.907554  
1959-06-30 23:59:59.999999999 -0.112548  2.033115  
1959-09-30 23:59:59.999999999 -0.767563 -0.901695  
1959-12-31 23:59:59.999999999  0.414969 -0.133802  
1960-03-31 23:59:59.999999999 -0.667184  0.172803  
1960-06-30 23:59:59.999999999 -0.272340 -0.750776  
1960-09-30 23:59:59.999999999  1.105903  0.414835  

[7 rows x 28 columns]

#### Pivoteamento do formato "largo" para o formato "longo"

Uma operação inversa de 'pivot' para DataFrames é 'pandas .melt'. Em vez de transformar uma coluna em várias em um novo DataFrame, fará o merge de várias colunas em uma só, gerando um DataFrame maior que a entrada.

In [118]:
df = pd.DataFrame({'key':['foo', 'bar', 'baz'], 
                   'A': [1, 2, 3], 
                   'B': [4, 5, 6], 
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


A coluna 'key' pode ser um indicador de grupo, e as outras colunas são valores de dados. Ao usar 'pandas.melt', devemos informar quais colunas (se houver) são indicadores de grupo. Usar 'key' como o único indicador de grupo nesse caso:

In [119]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


###### Usando 'pivot', podemos reformatar novamente e obter o layout original:

In [120]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


###### Como o resultado de 'pivot' cria um índice a partir da coluna usada como os rótulos das linhas, podemos usar 'reset_index' para passar os dados de volta para uma coluna:

In [121]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


###### Também é possível especificar um subconjunto de colunas a serem usadas como colunas de valores:

In [122]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


###### 'pandas.melt' também pode ser usado sem nenhum identificador de grupo:

In [123]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [124]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


8.4 Conclusão

Agora que dispomos de algum conhecimento básico do pandas para importação, limpeza e reorganização de dados, estamos prontos para prosseguir em direção à visualização de dados com a matplotlib.